<a href="https://colab.research.google.com/github/ppotnis-projects/repo/blob/main/NLP_RAG_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Data Description

The **Merck Manuals** are medical references published by the American pharmaceutical company Merck & Co., that cover a wide range of medical topics, including disorders, tests, diagnoses, and drugs. The manuals have been published since 1899, when Merck & Co. was still a subsidiary of the German company Merck.

The manual is provided as a PDF with over 4,000 pages divided into 23 sections.

## Installing and Importing Necessary Libraries and Dependencies

In [ ]:
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python==0.1.85 --force-reinstall --no-cache-dir -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 28.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 230.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 240.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 254.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 294.4 MB/s eta 0:00:00


In [ ]:
# For installing the libraries & downloading models from HF Hub
!pip install huggingface_hub==0.23.2 pandas==1.5.3 tiktoken==0.6.0 pymupdf==1.25.1 langchain==0.1.1 langchain-community==0.0.13 chromadb==0.4.22 sentence-transformers==2.3.1 numpy==1.25.2 -q


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.2/40.2 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.2/40.2 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 3.0 MB

In [ ]:
#Libraries for processing dataframes,text
import json,os
import tiktoken
import pandas as pd

#Libraries for Loading Data, Chunking, Embedding, and Vector Databases
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain_community.vectorstores import Chroma

from huggingface_hub import hf_hub_download
from llama_cpp import Llama

## Question Answering using LLM

### Downloading and Loading the model

In [ ]:
model_name_or_path = "TheBloke/Mistral-7B-Instruct-v0.2-GGUF"
model_basename = "mistral-7b-instruct-v0.2.Q6_K.gguf"

In [ ]:
model_path = hf_hub_download(
    repo_id= model_name_or_path,
    filename= model_basename
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


mistral-7b-instruct-v0.2.Q6_K.gguf:   0%|          | 0.00/5.94G [00:00<?, ?B/s]

In [ ]:
#model parameters when runtime is connected to GPU
llm = Llama(
    model_path=model_path,
    n_ctx=2300,
    n_gpu_layers=38,
    n_batch=512
)

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | 


### Response

In [ ]:
def response(query,max_tokens=750,temperature=0,top_p=0.95,top_k=50):
    model_output = llm(
      prompt=query,
      max_tokens=max_tokens,
      temperature=temperature,
      top_p=top_p,
      top_k=top_k
    )

    return model_output['choices'][0]['text']

In [ ]:
response("What treatment options are available for managing hypertension?")

'\n\nHypertension, or high blood pressure, is a common condition that can increase the risk of various health problems such as heart disease, stroke, and kidney damage. The good news is that there are several effective treatment options available to help manage hypertension and reduce the risk of complications. Here are some of the most commonly used treatments:\n\n1. Lifestyle modifications: Making lifestyle changes is often the first line of defense against hypertension. This may include eating a healthy diet rich in fruits, vegetables, whole grains, and lean proteins; limiting sodium intake; getting regular physical activity'

### Query 1: What is the protocol for managing sepsis in a critical care unit?

In [ ]:
user_input = "What is the protocol for managing sepsis in a critical care unit?"
response(user_input)

Llama.generate: prefix-match hit


'\n\nSepsis is a life-threatening condition that can arise from an infection, and it requires prompt recognition and aggressive management in a critical care unit. The following are general steps for managing sepsis in a critical care unit:\n\n1. Early recognition: Recognize the signs and symptoms of sepsis early and initiate treatment as soon as possible. Sepsis can present with various clinical features, including fever or hypothermia, tachycardia or bradycardia, altered mental status, respiratory distress, and lactic acidosis.\n2. Source control'

### Query 2: What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?

In [ ]:
user_input_2 = "What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?" #Complete the code to define the query #2
response(user_input_2)

Llama.generate: prefix-match hit


'\n\nAppendicitis is a medical condition characterized by inflammation of the appendix, a small tube-shaped organ located in the lower right side of the abdomen. The symptoms of appendicitis can vary from person to person, but some common signs include:\n\n1. Abdominal pain: The pain may start as a mild discomfort around the navel or in the lower right abdomen, which then gradually moves to the right lower quadrant and becomes more severe over time. The pain may be constant or intermittent and is often worsened by movement, coughing, or deep breathing'

### Query 3: What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?

In [ ]:
user_input_3 = "What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?" #Complete the code to define the query #3
response(user_input_3)

Llama.generate: prefix-match hit


"\n\nSudden patchy hair loss, also known as alopecia areata, is a common autoimmune disorder that affects the hair follicles. It can result in round or oval bald patches on the scalp, but it can also occur on other parts of the body such as the beard area, eyebrows, or eyelashes.\n\nThe exact cause of alopecia areata is not known, but it's believed to be related to a problem with the immune system. Some possible triggers for this condition include stress, genetics, viral infections, and certain medications."

### Query 4:  What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?

In [ ]:
user_input_4 = "What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?" #Complete the code to define the query #4
response(user_input_4)

Llama.generate: prefix-match hit


"\n\nA person who has sustained a physical injury to brain tissue, also known as a traumatic brain injury (TBI), may require various treatments depending on the severity and location of the injury. Here are some common treatments recommended for TBIs:\n\n1. Emergency care: The first priority is to ensure the person's airway is clear, they are breathing, and their heart is beating normally. In severe cases, emergency surgery may be required to remove hematomas or other obstructions.\n2. Medications: Depending on the symptoms, medications may be prescribed to manage conditions such as"

### Query 5: What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?

In [ ]:
user_input_5 = "What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?" #Complete the code to define the query #5
response(user_input_5)

Llama.generate: prefix-match hit


"\n\nFirst and foremost, if you suspect that someone has fractured their leg while hiking, it's essential to ensure their safety and prevent further injury. Here are some necessary precautions:\n\n1. Keep the person calm and still: Encourage them to remain as still as possible to minimize pain and prevent worsening the injury.\n2. Assess the situation: Check for any signs of shock, such as pale skin, rapid heartbeat, or shallow breathing. If you notice these symptoms, seek medical help immediately.\n3. Immobilize the leg: Use a splint, sl"

## Question Answering using LLM with Prompt Engineering

In [ ]:
system_prompt = """
  This AI tool is for informational purposes only and is not a substitute for professional medical advice. Answer the asked question as thoroughly and accurately as possible.

  When answering, please consider the following:
  General Information: Only if asked, provide a comprehensive overview of the medical condition or topic.
  Possible Causes/Risk Factors: Only if asked, explain the possible causes or risk factors associated with the condition.
  Symptoms/Signs: Only if asked, list common symptoms or signs that a person might experience.
  Diagnosis: Only if asked, explain comprehensively how the condition is typically diagnosed, including relevant tests or procedures.
  Treatment Options: If asked, describe general treatment approaches, including medications, therapies, and lifestyle changes.
  Complications: Mention potential complications associated with the condition.
  Patient Education: Provide general information on the condition in clear and concise language, suitable for a general audience.
  Further Reading: Include links to reputable sources, such as the National Institutes of Health (NIH), Mayo Clinic, and the Centers for Disease Control and Prevention (CDC), or professional organizations, to help the user find more information.
  Desired Format: Please format the response as a numbered list with a brief explanation for each section.
  Important Reminder: Provide this information assuming that a healthcare professional will review and use it; accuracy is of utmost importance.

  Keep the answer close to the context of the question asked and keep medical jargon to minimal.
"""

### Query 1: What is the protocol for managing sepsis in a critical care unit?

In [ ]:
user_input = system_prompt+"\n"+ "What is the protocol for managing sepsis in a critical care unit?"
response(user_input)

Llama.generate: prefix-match hit


"\n\n1. Recognition and Early Detection: Sepsis is a life-threatening condition that requires prompt recognition and intervention. The early detection of sepsis involves identifying patients who are at risk, recognizing signs and symptoms, and initiating appropriate treatment as soon as possible. (Source: National Institutes of Health)\n\n2. Initial Assessment: Once sepsis is suspected, perform a rapid assessment to determine the severity of illness and prioritize interventions. This may include assessing vital signs, level of consciousness, respiratory status, cardiovascular function, urine output, and lactate levels. (Source: Surviving Sepsis Campaign)\n\n3. Fluid Resuscitation: Intravenous fluids are typically the first line of treatment for sepsis. The goal is to maintain adequate tissue perfusion and prevent organ dysfunction. Aggressive fluid resuscitation may be required, with targets for mean arterial pressure (MAP) and central venous pressure (CVP) based on the patient's clini

### Query 2: What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?

In [ ]:
user_input_2 = system_prompt+"\n"+"What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?"
response(user_input_2)

Llama.generate: prefix-match hit


"\n\n1. Symptoms/Signs:\n   - Sudden pain in the lower right abdomen that starts near the navel and moves to the right side.\n   - Loss of appetite.\n   - Nausea and vomiting.\n   - Abdominal swelling.\n   - Fever (often over 101 degrees Fahrenheit or 38.3 degrees Celsius).\n   - Pain when walking, coughing, or making other jarring movements.\n   - Inability to pass gas or have a bowel movement.\n2. Diagnosis:\n   Appendicitis is typically diagnosed based on a combination of symptoms, physical examination findings, and imaging studies such as ultrasound or CT scan. A healthcare professional will perform a thorough medical history and physical exam, paying close attention to the abdomen. If appendicitis is suspected, further testing with imaging studies may be ordered to confirm the diagnosis.\n3. Treatment Options:\n   - Antibiotics alone are not sufficient to treat appendicitis as they do not address the inflamed appendix itself. Surgical removal of the appendix (appendectomy) is the 

### Query 3: What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?

In [ ]:
user_input_3 = system_prompt+"\n"+"What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?" #Complete the code to pass the query #3
response(user_input_3)

Llama.generate: prefix-match hit


'\n\n1. Possible Causes/Risk Factors:\n   - Androgenetic alopecia (male or female pattern baldness)\n   - Alopecia areata (an autoimmune condition that causes hair loss in patches)\n   - Trauma, stress, or physical damage to the scalp\n   - Nutritional deficiencies, such as iron-deficiency anemia or protein deficiency\n   - Certain medications, including antidepressants and chemotherapy drugs\n   - Hormonal imbalances, such as thyroid disorders\n   - Infections, such as ringworm of the scalp\n2. Symptoms/Signs:\n   - Round or oval bald patches on the scalp\n   - Sudden onset of hair loss in localized areas\n   - Hair loss may occur in other parts of the body, such as eyebrows or beard\n3. Diagnosis:\n   - A thorough medical history and physical examination\n   - Blood tests to check for nutritional deficiencies or hormonal imbalances\n   - Scalp biopsy to rule out conditions like alopecia areata or ringworm\n4. Treatment Options:\n   - Topical treatments, such as minoxidil (Rogaine) or

### Query 4:  What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?

In [ ]:
user_input_4 = system_prompt+'\n'+'What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?' #Complete the code to pass the query #4
response(user_input_4)

Llama.generate: prefix-match hit


'\n\n1. Rehabilitation: The primary treatment for brain injuries is rehabilitation, which can help restore lost functions and improve overall quality of life. This may include occupational therapy, speech therapy, physical therapy, and cognitive rehabilitation.\n2. Medications: Depending on the specific symptoms and conditions, various medications may be prescribed to manage symptoms such as pain, seizures, or depression. For instance, anti-inflammatory drugs, anticonvulsants, and antidepressants might be used.\n3. Surgery: In some cases, surgery may be necessary to remove hematomas (blood clots) or repair skull fractures that could be causing further damage or impairment.\n4. Assistive Devices: Depending on the extent of the injury and resulting impairments, assistive devices such as wheelchairs, walkers, braces, or communication aids may be recommended to help individuals regain independence and improve their daily functioning.\n5. Lifestyle modifications: Making lifestyle changes ca

### Query 5: What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?

In [ ]:
user_input_5 = system_prompt+'\n'+'What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?' #Complete the code to pass the query #5
response(user_input_5)

Llama.generate: prefix-match hit


"\n\n1. **Immediate Care:** If a person suspects they have fractured their leg while hiking, they should try to keep the injured leg still to prevent further damage. They should not attempt to walk on it and should seek medical attention as soon as possible. If the injury is severe or if the person cannot move, call for emergency medical services.\n\n2. **Precautions:** While waiting for medical help, the person should keep the injured leg elevated above heart level to reduce swelling and pain. They should also apply a cool compress to the area to help reduce inflammation. If there is visible bleeding, use a clean cloth to apply gentle pressure to the wound to control the bleeding.\n\n3. **Diagnosis:** A healthcare professional will typically diagnose a leg fracture through a physical examination and imaging tests such as X-rays or CT scans. They may also ask about the circumstances of the injury and perform other diagnostic tests if necessary.\n\n4. **Treatment Options:** Treatment fo

## Data Preparation for RAG

### Loading the Data

In [ ]:
 from google.colab import drive
 drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
manual_pdf_path = "/content/drive/MyDrive/Colab Notebooks/medical_diagnosis_manual.pdf"

In [ ]:
pdf_loader = PyMuPDFLoader(manual_pdf_path)

In [ ]:
manual = pdf_loader.load()

### Data Overview

#### Checking the first 5 pages

In [ ]:
for i in range(5):
    print(f"Page Number : {i+1}",end="\n")
    print(manual[i].page_content,end="\n")

Page Number : 1
poojakpotnis@gmail.com
Z6SR9UQAV3
ant for personal use by poojakpotnis@g
shing the contents in part or full is liable 

Page Number : 2
poojakpotnis@gmail.com
Z6SR9UQAV3
This file is meant for personal use by poojakpotnis@gmail.com only.
Sharing or publishing the contents in part or full is liable for legal action.

Page Number : 3
Table of Contents
1
Front    ................................................................................................................................................................................................................
1
Cover    .......................................................................................................................................................................................................
2
Front Matter    .......................................................................................................................................................................................

#### Checking the number of pages

In [ ]:
len(manual)

4114

### Data Chunking

In [ ]:
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    encoding_name='cl100k_base',
    chunk_size=512,
    chunk_overlap= 100
)

In [ ]:
document_chunks = pdf_loader.load_and_split(text_splitter)

In [ ]:
len(document_chunks)

9117

In [ ]:
document_chunks[0].page_content

'poojakpotnis@gmail.com\nZ6SR9UQAV3\nant for personal use by poojakpotnis@g\nshing the contents in part or full is liable'

In [ ]:
document_chunks[233].page_content

'relatively low muscle-to-fat mass ratio), increases risk. The syndrome is less common among people who\nhave excess subcutaneous fat around the hips (called pear shape) and a low waist-to-hip ratio (reflecting\na higher muscle-to-fat mass ratio).\nExcess abdominal fat leads to excess free fatty acids in the portal vein, increasing fat accumulation in the\nliver. Fat also accumulates in muscle cells. Insulin resistance develops, with hyperinsulinemia. Glucose\nmetabolism is impaired, and dyslipidemias and hypertension develop. Serum uric acid levels are typically\nelevated, and a prothrombotic state (with increased levels of fibrinogen and plasminogen activator\ninhibitor I) and an inflammatory state develop. Patients have an increased risk of obstructive sleep apnea.\nOther risks include nonalcoholic steatohepatitis, chronic kidney disease, polycystic ovary syndrome (for\nwomen), and low serum testosterone, erectile dysfunction, or both (for men).\nDiagnosis\n• Waist circumference and

In [ ]:
document_chunks[232].page_content

'Mood and work and personal relationships usually improve.\nLong-term follow-up data are not yet available because these procedures are relatively new.\nFollow-up\nPatients should be monitored every 4 to 6 wk while weight loss is rapid (usually about the first 6 mo after\nsurgery), then every 6 to 12 mo. Weight and BP are checked, and eating habits are reviewed. Blood tests\n(usually CBC, electrolytes, glucose, BUN, creatinine, albumin, and protein) and liver function tests are\ndone at each visit. If alkaline phosphatase is increased, parathyroid hormone level is measured; if\nparathyroid hormone level is abnormal, bone density is monitored. If weight loss exceeds about 9 kg (20\nlb)/mo, visits should be scheduled at least monthly, and blood tests should include Mg, phosphorus,\nvitamin B12, and iron levels. Nutritional supplementation is sometimes necessary.\nMetabolic Syndrome\nMetabolic syndrome (syndrome X, insulin resistance syndrome) is characterized by a clustering\nof risk fac

As expected, there are some overlaps

### Embedding

In [ ]:
embedding_model = SentenceTransformerEmbeddings(model_name="thenlper/gte-small")

modules.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/68.1k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/66.7M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/394 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
embedding_1 = embedding_model.embed_query(document_chunks[0].page_content)
embedding_2 = embedding_model.embed_query(document_chunks[1].page_content)

In [ ]:
print("Dimension of the embedding vector ",len(embedding_1))
len(embedding_1)==len(embedding_2)

Dimension of the embedding vector  384


True

In [ ]:
embedding_1,embedding_2

([0.017077315598726273,
  -0.017865756526589394,
  0.007138863205909729,
  0.009941055439412594,
  0.06734592467546463,
  0.033118490129709244,
  0.022149337455630302,
  0.010570342652499676,
  -0.03127063065767288,
  -0.022957071661949158,
  -0.01635400764644146,
  0.00695371488109231,
  -0.06326551735401154,
  0.03473808988928795,
  -0.015315532684326172,
  0.06324003636837006,
  0.05961177125573158,
  -0.00020239669538568705,
  0.011081624776124954,
  -0.021002382040023804,
  0.011395290493965149,
  -0.02372860722243786,
  0.0041591702029109,
  -0.003381456481292844,
  -0.008261062204837799,
  0.0016110154101625085,
  0.02116435021162033,
  0.051422469317913055,
  -0.0988711267709732,
  -0.010602331720292568,
  0.0470467172563076,
  0.006477430462837219,
  -0.0035742074251174927,
  -0.02306084707379341,
  -0.01122251059859991,
  -0.041595231741666794,
  -0.018712570890784264,
  -0.004566507879644632,
  -0.04843727499246597,
  0.004063866566866636,
  -0.015456149354577065,
  -0.01827

### Vector Database

In [ ]:
out_dir = 'medical_db'

if not os.path.exists(out_dir):
  os.makedirs(out_dir)

In [ ]:
vectorstore = Chroma.from_documents(
    document_chunks,
    embedding_model,
    persist_directory=out_dir
)

In [ ]:
vectorstore = Chroma(persist_directory=out_dir,embedding_function=embedding_model)

In [ ]:
vectorstore.embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False})
  (2): Normalize()
), model_name='thenlper/gte-small', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False)

In [ ]:
vectorstore.similarity_search("Can you explain the nuances of hypertension?",k=3)

[Document(page_content='Chapter 208. Arterial Hypertension\nIntroduction\nHypertension is sustained elevation of resting systolic BP (≥ 140 mm Hg), diastolic BP (≥ 90 mm\nHg), or both. Hypertension with no known cause (primary; formerly, essential hypertension) is\nmost common. Hypertension with an identified cause (secondary hypertension) is usually due\nto a renal disorder. Usually, no symptoms develop unless hypertension is severe or long-\nstanding. Diagnosis is by sphygmomanometry. Tests may be done to determine cause, assess\ndamage, and identify other cardiovascular risk factors. Treatment involves lifestyle changes and\ndrugs, including diuretics, β-blockers, ACE inhibitors, angiotensin II receptor blockers, and Ca\nchannel blockers.\nIn the US, about 65 million people have hypertension. Only about 70% of these people are aware that\nthey have hypertension, only 59% are being treated, and only 34% have adequately controlled BP. In\nadults, hypertension occurs more often in blac

### Retriever

In [ ]:
retriever = vectorstore.as_retriever(
    search_type='similarity',
    search_kwargs={'k': 2}

In [ ]:
rel_docs = retriever.get_relevant_documents("What are the symptoms of skin disorders and their treatment options?")
rel_docs

[Document(page_content='skin disorders, drugs, cancer, or unknown causes. Symptoms and signs are pruritus, diffuse\nerythema, and epidermal sloughing. Diagnosis is clinical. Treatment involves corticosteroids\nand correction of the cause.\nExfoliative dermatitis is a manifestation of rapid epidermal cell turnover. Its cause is unknown, but it most\noften occurs in the context of preexisting skin disorders (eg, atopic dermatitis, contact dermatitis,\nseborrheic dermatitis, psoriasis, pityriasis rubra pilaris), use of drugs (eg, penicillin, sulfonamides,\nisoniazid, phenytoin, barbiturates), and cancer (eg, mycosis fungoides, leukemia, and, rarely, ade-\nnocarcinomas). Up to 25% of patients have no identifiable underlying disease.\nSymptoms and Signs\nSymptoms include pruritus, malaise, and chills. Diffuse erythema initially occurs in patches but spreads\nand involves all or nearly all of the body. Extensive epidermal sloughing leads to abnormal\nthermoregulation, nutritional deficiencie

In [ ]:
model_output = llm(
      "What are the common ailments with respect to excessive sun exposure?",
      max_tokens=500,
      temperature=0,
    )

In [ ]:
model_output['choices'][0]['text']

"\n\nExcessive sun exposure can lead to several health issues. Some of the most common ailments associated with prolonged sun exposure include:\n\n1. Sunburn: This is the most common and immediate effect of excessive sun exposure. Sunburn appears as red, painful, and swollen skin that may blister or peel. Severe sunburns can lead to dehydration, fever, and even hospitalization in extreme cases.\n2. Heat Exhaustion: Prolonged exposure to the sun can also lead to heat exhaustion, which is characterized by symptoms such as dizziness, fatigue, headache, rapid heartbeat, and profuse sweating. In severe cases, heat exhaustion can progress to heat stroke, a potentially life-threatening condition.\n3. Skin Damage: Long-term exposure to the sun can cause premature aging of the skin, leading to wrinkles, fine lines, age spots, and other signs of aging. It can also increase the risk of developing skin cancer, including melanoma, basal cell carcinoma, and squamous cell carcinoma.\n4. Eye Damage: P

The above response is somewhat generic and is solely based on the data the model was trained on, rather than the medical manual.  

Let's now provide our own context.

### System and User Prompt Template

Prompts guide the model to generate accurate responses. Here, we define two parts:

    1. The system message describing the assistant's role.
    2. A user message template including context and the question.

In [ ]:
qna_system_message = """
You are a medical assistant whose job is to review the report and provide a detailed response from the context.
When answering, please provide detailed general information based from the context.
User input will have context required by you to answer the questions.
This context will begin with the token: ###Context.
The context contains references to specific portions of a document relevant to the user query.

User questions will begin with the token: ###Question.

Please answer only using the context provided in the input. Do not mention anything about the context in your final answer.

If the answer is not found in the context, respond with "I do not know".
"""

In [ ]:
qna_user_message_template = """
###Context
Here are some documents that are relevant to the question mentioned below.
{context}

###Question
{question}
"""

### Response Function

In [ ]:
def generate_rag_response(user_input,k=3,max_tokens=128,temperature=0,top_p=0.95,top_k=50):
    global qna_system_message,qna_user_message_template
    # Retrieve relevant document chunks
    relevant_document_chunks = retriever.get_relevant_documents(query=user_input,k=k)
    context_list = [d.page_content for d in relevant_document_chunks]

    # Combine document chunks into a single context
    context_for_query = ". ".join(context_list)

    user_message = qna_user_message_template.replace('{context}', context_for_query)
    user_message = user_message.replace('{question}', user_input)

    prompt = qna_system_message + '\n' + user_message

    # Generate the response
    try:
        response = llm(
                  prompt=prompt,
                  max_tokens=max_tokens,
                  temperature=temperature,
                  top_p=top_p,
                  top_k=top_k
                  )

        # Extract and print the model's response
        response = response['choices'][0]['text'].strip()
    except Exception as e:
        response = f'Sorry, I encountered the following error: \n {e}'

    return response

## Question Answering using RAG

### Query 1: What is the protocol for managing sepsis in a critical care unit?

In [ ]:
user_input = "What is the protocol for managing sepsis in a critical care unit?"
generate_rag_response(user_input)

Llama.generate: prefix-match hit


'Based on the context provided, the protocol for managing sepsis in a critical care unit includes:\n1. Fluid resuscitation with 0.9% normal saline until CVP reaches 8 mm Hg (10 cm H2O) or PAOP reaches 12 to 15 mm Hg. Oliguria with hypotension is not a contraindication to vigorous fluid resuscitation.\n2. Administration of oxygen through mask or nasal prongs, and possible tracheal intubation and mechanical ventilation for respiratory failure.\n3.'

### Query 2: What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?

In [ ]:
user_input_2 = "What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?"
generate_rag_response(user_input_2)

Llama.generate: prefix-match hit


"The common symptoms for appendicitis include epigastric or periumbilical pain followed by brief nausea, vomiting, anorexia, and after a few hours, the pain shifts to the right lower quadrant. Pain increases with cough and motion. Classic signs are right lower quadrant direct and rebound tenderness located at McBurney's point (junction of the middle and outer thirds of the line joining the umbilicus to the anterior superior spine). Additional signs include pain felt in the right lower quadrant with palpation of the left lower quadrant (Rov"

### Query 3: What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?

In [ ]:
user_input_3 = "What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?" #Complete the code to pass the query #3
generate_rag_response(user_input_3)

Llama.generate: prefix-match hit


'Based on the context provided, sudden patchy hair loss, also known as alopecia areata, can be treated with various methods depending on the severity of the condition. For mild cases, topical treatments such as minoxidil or anthralin may be effective. Intralesional corticosteroid injections or systemic corticosteroids may be used for more severe cases. Topical immunotherapy using diphencyprone or squaric acid dibutylester can also be an option. Psoralen plus ultraviolet A (PUVA) therapy is another treatment'

### Query 4:  What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?

In [ ]:
user_input_4 = "What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?"
generate_rag_response(user_input_4)

Llama.generate: prefix-match hit


'Based on the context provided, the initial treatment for a person with traumatic brain injury (TBI) includes ensuring a reliable airway and maintaining adequate ventilation, oxygenation, and blood pressure. Surgery may be needed in patients with more severe injuries to place monitors to track and treat intracranial pressure, decompress the brain if intracranial pressure is increased, or remove intracranial hematomas. In the first few days after the injury, maintaining adequate brain perfusion and oxygenation and preventing complications of altered sensorium are important. Subsequently, many patients require rehabilitation.'

### Query 5: What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?

In [ ]:
user_input_5 = "What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?" #Complete the code to pass the query #5
generate_rag_response(user_input_5)

Llama.generate: prefix-match hit


'Based on the context provided, here is a detailed response:\n\nA person who has fractured their leg during a hiking trip should first seek medical attention as soon as possible. In the meantime, they can take some initial steps to help manage their injury and prevent further complications. These steps include:\n\n1. Immobilize the affected limb using a splint or sling to prevent any movement that could worsen the fracture.\n2. Apply ice packs intermittently to reduce swelling and pain.\n3. Elevate the injured leg above heart level whenever possible to help reduce'

### Fine-tuning

#### Query 1: What is the protocol for managing sepsis in a critical care unit?

In [ ]:
user_input = "What is the protocol for managing sepsis in a critical care unit?"
generate_rag_response(user_input,k=3,max_tokens=500,temperature=0.2,top_k=20)

Llama.generate: prefix-match hit


"The protocol for managing sepsis in a critical care unit involves several steps. First, fluid resuscitation with 0.9% normal saline should be given until central venous pressure (CVP) reaches 8 mm Hg (10 cm H2O) or pulmonary artery occlusive pressure (PAOP) reaches 12 to 15 mm Hg. Oliguria with hypotension is not a contraindication to vigorous fluid resuscitation, and the quantity of fluid required often exceeds the normal blood volume and may reach 10 L over 4 to 12 hours. If a patient remains hypotensive after CVP or PAOP has been raised to target levels, dopamine may be given to increase mean arterial pressure (BP) to at least 60 mm Hg. If the dopamine dose exceeds 20 μg/kg/min, another vasopressor, typically norepinephrine, may be added. However, vasoconstriction caused by higher doses of dopamine and norepinephrine poses risks of organ hypoperfusion and acidosis, and these drugs have not been shown to improve survival. Oxygen should be given by mask or nasal prongs, and tracheal 

#### Query 2: What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?

In [ ]:
user_input_2 = "What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?"
generate_rag_response(user_input_2,k=5,max_tokens=800,temperature=0.5,top_k=50)

Llama.generate: prefix-match hit


"Symptoms for appendicitis include epigastric or periumbilical pain followed by brief nausea, vomiting, anorexia, and after a few hours, the pain shifts to the right lower quadrant. Pain increases with cough and motion. Classic signs are right lower quadrant direct and rebound tenderness located at McBurney's point (junction of the middle and outer thirds of the line joining the umbilicus to the anterior superior spine). Additional signs are pain felt in the right lower quadrant with palpation of the left lower quadrant (Rovsing sign), an increase in pain from passive extension of the right hip joint that stretches the iliopsoas muscle (psoas sign), or pain caused by passive internal rotation of the flexed thigh (obturator sign). Low-grade fever is common. However, these classic findings appear in less than 50% of patients, and many variations of symptoms and signs occur. Pain may not be localized, particularly in infants and children. Tenderness may be diffuse or absent. Bowel movemen

#### Query 3: What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?

In [ ]:
user_input_3 = "What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?" #Complete the code to pass the query #3
generate_rag_response(user_input_3,k=2,max_tokens=328,temperature=1)

Llama.generate: prefix-match hit


'Based on the context, sudden patchy hair loss, also known as alopecia areata, is thought to be an autoimmune disorder affecting genetically susceptible people exposed to unclear environmental triggers. The scalp and beard are most frequently affected areas. The effective treatments or solutions for addressing this condition include topical, intralesional, or systemic corticosteroids, topical minoxidil, topical anthralin, topical immunotherapy (diphencyprone or squaric acid dibutylester), or psoralen plus ultraviolet A (PUVA). Hormonal modulators such as oral contraceptives or spironolactone may be useful for female-pattern hair loss associated with hyperandrogenemia. It is important to note that treatment should only be initiated after a thorough evaluation by a healthcare professional to rule out any underlying disorders. The context also mentions that scarring alopecia as seen in central centrifugal scarring alopecia, dissecting cellulitis of the scalp, and acne keloidalis nuchae is

#### Query 4:  What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?

In [ ]:
user_input_4 = "What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?"
generate_rag_response(user_input_4,k=2,max_tokens=1000,temperature=0.3,top_k=10)

Llama.generate: prefix-match hit


"Based on the context provided, the initial treatment for a person with a traumatic brain injury (TBI) includes ensuring a reliable airway and maintaining adequate ventilation, oxygenation, and blood pressure. Surgery may be needed in patients with more severe injuries to place monitors to track and treat intracranial pressure, decompress the brain if intracranial pressure is increased, or remove intracranial hematomas. In the first few days after the injury, maintaining adequate brain perfusion and oxygenation and preventing complications of altered sensorium are important. Subsequently, many patients require rehabilitation. The Merck Manual also mentions that for patients whose coma exceeds 24 hours, a prolonged period of rehabilitation is often required. Rehabilitation is best provided through a team approach that includes physical, occupational, and speech therapy, skill-building activities, and counseling to meet the patient's social and emotional needs. Brain injury support group

#### Query 5: What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?

In [ ]:
user_input_5 = "What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?"
generate_rag_response(user_input_5,k=3,max_tokens=800,temperature=2)

Llama.generate: prefix-match hit


"Based on the context provided, here's the detailed response:\n\nInitially after a fractured leg from a hiking trip, it is important to ensure the affected limb is not subjected to weight bearing activities. The patient should be encouraged to rest and avoid any movements that may cause discomfort or pain in the affected area. If possible, a splint or a cast may be applied to immobilize the fractured bone and maintain proper alignment until further evaluation and treatment by a healthcare professional is sought.\n\nDuring the recovery process, there are specific precautions that should be taken to ensure proper care for the injured leg. While rehabilitation varies depending on the type of surgery, for most fractures full weight bearing may not begin until a few days after surgery, if at all. Exercise activities aimed at increasing strength and preventing atrophy in both legs should commence as soon as possible. These exercises can consist of isometric movements during the initial stage

## Output Evaluation

Let us now use the LLM-as-a-judge method to check the quality of the RAG system on two parameters - retrieval and generation.

- We are using the same Mistral model for evaluation, so basically here the llm is rating itself on how well he has performed in the task.

In [ ]:
groundedness_rater_system_message = """
You are tasked with rating AI generated answers to questions posed by users.
You will be presented a question, context used by the AI system to generate the answer and an AI generated answer to the question.
In the input, the question will begin with ###Question, the context will begin with ###Context while the AI generated answer will begin with ###Answer.

Evaluation criteria:
The task is to judge the extent to which the metric is followed by the answer.
1 - The metric is not followed at all
2 - The metric is followed only to a limited extent
3 - The metric is followed to a good extent
4 - The metric is followed mostly
5 - The metric is followed completely

Metric:
The answer should be derived only from the information presented in the context

Instructions:
1. First write down the steps that are needed to evaluate the answer as per the metric.
2. Give a step-by-step explanation if the answer adheres to the metric considering the question and context as the input.
3. Next, evaluate the extent to which the metric is followed.
4. Use the previous information to rate the answer using the evaluaton criteria and assign a score.
"""

In [ ]:
relevance_rater_system_message = """
You are tasked with rating AI generated answers to questions posed by users.
You will be presented a question, context used by the AI system to generate the answer and an AI generated answer to the question.
In the input, the question will begin with ###Question, the context will begin with ###Context while the AI generated answer will begin with ###Answer.

Evaluation criteria:
The task is to judge the extent to which the metric is followed by the answer.
1 - The metric is not followed at all
2 - The metric is followed only to a limited extent
3 - The metric is followed to a good extent
4 - The metric is followed mostly
5 - The metric is followed completely

Metric:
Relevance measures how well the answer addresses the main aspects of the question, based on the context.
Consider whether all and only the important aspects are contained in the answer when evaluating relevance.

Instructions:
1. First write down the steps that are needed to evaluate the context as per the metric.
2. Give a step-by-step explanation if the context adheres to the metric considering the question as the input.
3. Next, evaluate the extent to which the metric is followed.
4. Use the previous information to rate the context using the evaluaton criteria and assign a score.
"""

In [ ]:
user_message_template = """
###Question
{question}

###Context
{context}

###Answer
{answer}
"""

In [ ]:
def generate_ground_relevance_response(user_input,k=3,max_tokens=128,temperature=0,top_p=0.95,top_k=50):
    global qna_system_message,qna_user_message_template
    # Retrieve relevant document chunks
    relevant_document_chunks = retriever.get_relevant_documents(query=user_input,k=3)
    context_list = [d.page_content for d in relevant_document_chunks]
    context_for_query = ". ".join(context_list)

    # Combine user_prompt and system_message to create the prompt
    prompt = f"""[INST]{qna_system_message}\n
                {'user'}: {qna_user_message_template.format(context=context_for_query, question=user_input)}
                [/INST]"""

    response = llm(
            prompt=prompt,
            max_tokens=max_tokens,
            temperature=temperature,
            top_p=top_p,
            top_k=top_k,
            stop=['INST'],
            echo=False
            )

    answer =  response["choices"][0]["text"]

    # Combine user_prompt and system_message to create the prompt
    groundedness_prompt = f"""[INST]{groundedness_rater_system_message}\n
                {'user'}: {user_message_template.format(context=context_for_query, question=user_input, answer=answer)}
                [/INST]"""

    # Combine user_prompt and system_message to create the prompt
    relevance_prompt = f"""[INST]{relevance_rater_system_message}\n
                {'user'}: {user_message_template.format(context=context_for_query, question=user_input, answer=answer)}
                [/INST]"""

    response_1 = llm(
            prompt=groundedness_prompt,
            max_tokens=max_tokens,
            temperature=temperature,
            top_p=top_p,
            top_k=top_k,
            stop=['INST'],
            echo=False
            )

    response_2 = llm(
            prompt=relevance_prompt,
            max_tokens=max_tokens,
            temperature=temperature,
            top_p=top_p,
            top_k=top_k,
            stop=['INST'],
            echo=False
            )

    return response_1['choices'][0]['text'],response_2['choices'][0]['text']

### Query 1: What is the protocol for managing sepsis in a critical care unit?

In [ ]:
ground,rel = generate_ground_relevance_response(user_input="What is the protocol for managing sepsis in a critical care unit?",max_tokens=370)

print(ground,end="\n\n")
print(rel)

Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit


 Steps to evaluate the answer:
1. Identify the information in the context related to managing sepsis in a critical care unit.
2. Compare the information in the context with the AI generated answer to check if all the details are present and accurate.
3. Verify that the answer is derived only from the information presented in the context.

Explanation:
The AI generated answer includes all the necessary steps for managing sepsis in a critical care unit as outlined in the context. The answer accurately mentions fluid resuscitation, broad-spectrum antibiotics, drainage of abscesses and excision of necrotic tissue, normalization of blood glucose levels, replacement-dose corticosteroids, monitoring of various physiologic parameters, urine output measurement, and use of vasopressors if needed.

Evaluation:
The metric is followed completely as the answer is derived only from the information presented in the context.

Rating:
Based on the evaluation criteria, I would rate this answer a 5 for fo

### Query 2: What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?

In [ ]:
ground,rel = generate_ground_relevance_response(user_input="What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?",max_tokens=500, temperature=1, k=5) #Complete the code to pass the query #2 along with parameters if needed

print(ground,end="\n\n")
print(rel)

Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit


 Steps for evaluation:
1. Identify the key information about appendicitis symptoms and treatment from the context.
2. Compare the identified information with the AI generated answer to check if all elements come from the context.
3. Evaluate the extent to which the AI answer sticks to the metric by determining if it only references the context provided.

Explanation:
The AI answer correctly lists the common symptoms of appendicitis, including epigastric or periumbilical pain followed by brief nausea, vomiting, and anorexia, as well as right lower quadrant pain that increases with cough and motion and is accompanied by direct and rebound tenderness at McBurney's point. It also mentions other possible signs like Rovsing sign, psoas sign, obturator sign, and a low-grade fever. Furthermore, the answer explains that appendicitis cannot be cured through medicine and requires surgical removal for proper treatment.

The information provided in the AI answer is entirely derived from the context

### Query 3: What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?

In [ ]:
ground,rel = generate_ground_relevance_response(user_input="What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?",k=5,max_tokens=500,top_k=10) #Complete the code to pass the query #3 along with parameters if needed

print(ground,end="\n\n")
print(rel)

Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit


 Steps to evaluate the answer:
1. Identify the information in the context related to effective treatments or solutions for sudden patchy hair loss (alopecia areata).
2. Check if the AI generated answer only includes information from the context and not any additional or irrelevant details.
3. Verify that the causes of alopecia areata mentioned in the answer are consistent with the context.

The AI generated answer adheres to the metric as it only includes information derived from the context about effective treatments for sudden patchy hair loss (alopecia areata) and its possible causes. Therefore, the extent to which the metric is followed is 5 - The metric is followed completely.

Based on the evaluation criteria:
1 - The metric is not followed at all
2 - The metric is followed only to a limited extent
3 - The metric is followed to a good extent
4 - The metric is followed mostly
5 - The metric is followed completely

Since the answer follows the metric completely, it would receive a 

### Query 4: What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?

In [ ]:
ground,rel = generate_ground_relevance_response(user_input="What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?",k=10,max_tokens=800,temperature=1) #Complete the code to pass the query #4 along with parameters if needed

print(ground,end="\n\n")
print(rel)

Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit


 Steps for evaluation:
1. Identify the key information related to treatments for traumatic brain injury from the context.
2. Compare the identified information with the content of the AI generated answer.
3. Determine if the AI generated answer is derived only from the information presented in the context.

Explanation:
The AI generated answer mentions the initial treatment for a person with a traumatic brain injury (TBI) which includes ensuring a reliable airway, maintaining adequate ventilation, oxygenation, and blood pressure. It also states that surgery may be needed in cases with more severe injuries and rehabilitation is required subsequently. Furthermore, it suggests a team approach to rehabilitation that combines physical, occupational, and speech therapy, skill-building activities, and counseling.

The context of the question includes information on initial treatment for TBI which involves ensuring a reliable airway, maintaining adequate ventilation, oxygenation, and blood pre

### Query 5: What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?

In [ ]:
ground,rel = generate_ground_relevance_response(user_input="What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?",k=7,max_tokens=800,top_p=0.80,top_k=30) #Complete the code to pass the query #5 along with parameters if needed

print(ground,end="\n\n")
print(rel)

Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit


 Steps to evaluate the answer:
1. Identify if the answer directly addresses the question about necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip.
2. Check if the information in the answer is derived only from the context provided.

The answer does not directly address the question about the necessary precautions and treatment steps specifically for someone who has fractured their leg during a hiking trip. However, it does provide some relevant information about hip surgery rehabilitation and care for patients with neuropathic foot ulcers. The answer goes beyond the context by adding recommendations for immobilization, seeking medical attention, following healthcare provider's recommendations for rehabilitation exercises, maintaining good nutrition and hydration, and wearing appropriate orthotic protection or special shoes if necessary.

Since the question asked about precautions and treatment steps for a person who has fractured the